In [1]:
# Importing Qiskit
from qiskit import *
from qiskit.qasm2 import dumps
from qiskit_aer import Aer

import base64
import numpy as np
# import operation as op
from functools import reduce
import random
from random import randrange

# import warnings
# warnings.filterwarnings("ignore")


# 1. Qubit Manipulation

In [2]:
def NoisyChannel(qc1, qc2, qc1_name, errors, noise = 5e-4):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates), simulates a noisy quantum channel where Pauli errors (X - bit flip; Z - phase flip)
        will occur in qc2, and then initializes another circuit qc2 with the introduced noise.
    ''' 
    
    # Retrieve quantum state from qasm code of qc1
    qs = [dumps(qc1[i]).split('\n') for i in range(len(qc1))]
    
    # Process the code to get the instructions
    parsed_instructions = []
    for i, qasm_code in enumerate(qs):
        for line in qasm_code:
            line = line.strip()    # removing leading/trailing whitespace
            if line.startswith(('x', 'h', 'measure')):
                line = line.replace('0', str(i))
                parsed_instructions.append(line)
    
    # Apply parsed instructions to qc2
    for instruction in parsed_instructions:
        if instruction.startswith('x'):
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].x(0)
            
        elif instruction.startswith('h'):
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].h(0)
        
        elif instruction.startswith('measure'):
            continue    # exclude measuring
            
        else:
            print(f"Unable to parse instruction: {instruction}")
            raise Exception('Unable to parse instruction')
    
    # Introducing noise (taking input)
    for instruction in parsed_instructions:
        if random.random() < noise:
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].x(0)     # Apply bit-flip error
            errors[0] += 1
            
        if random.random() < noise:
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].z(0)     # Apply phase-flip error
            errors[1] += 1

    return errors

In [3]:
def generate_random_bits(num):
    """This function generates a random array of bits(0/1) of size = num"""
    # bits = np.array([random.randint(0, 1) for _ in range(num)])    # Randomly fills the array with 0/1

    bit_string = ""
    for _ in range(num):
        rand_bit = random.randint(0, 1)     # Flip Coin
        bit_string += str(rand_bit)
        
    return bit_string

In [4]:
def generate_random_bases(num_of_bases):
    """This function selects a random basis for each bit"""
    
    bases_string = ""
    for _ in range(num_of_bases):
        randBasis = random.randint(0, 1)     # Flip Coin

        if randBasis == 0:
            bases_string += "Z" 
        else:
            bases_string += "X"
            
    return bases_string

In [5]:
def encode(bits, bases):
    """This function encodes each bit into the given basis."""
    
    encoded_qubits = []
    
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)     # Create a quantum circuit for each qubit
        
        # Possible Cases
        if bit == "1" :
            qc.x(0)

        if basis == 'X' :
            qc.h(0)
            
        encoded_qubits.append(qc)
            
    return encoded_qubits

In [6]:
def measure(qubits, bases):
    """This function measures each qubit in the corresponding basis chosen for it.
        - qubits : a series of 1-qubit Quantum Circuit
        - bases : a string of random [X, Z] bases"""

    # bits = np.zeros(len(bases), dtype = int)    # The results of measurements
    bits = ""
        
    for idx, (qubit, basis) in enumerate(zip(qubits, bases)):

        if basis == "X" :
            qubit.h(0)
            
        qubit.measure(0, 0)
        
        # Execute on Simulator
        simulator = Aer.get_backend('qasm_simulator')
        transpiled_circuit = transpile(qubit, simulator)
        result = simulator.run(transpiled_circuit, shots=1).result()
        counts = result.get_counts()
        measured_bit = max(counts, key=counts.get)     # Max doesn't matter for simulator since there is only one shot.

        bits += str(measured_bit)
        # bits[idx] = int(measured_bit)
        
    return bits

In [7]:
def array_to_string(array):
    result = np.array2string(
        array, 
        separator = "", 
        max_line_width = (len(array)+3))
    return result.strip('[').strip(']')


In [8]:
def convert_to_octets(key):

    octets = []
    num_octets = len(key) // 8

    for i in range(num_octets):
        start = i * 8
        end = start + 8
        octet = key[start:end]
        octets.append(int(octet, 2))

    return bytearray(octets)

# 2. Preparation

In [9]:
### Use the function Order(bits) to get 'order'.

In [10]:
%run ./Hamming.ipynb    # For error correction

In [11]:
# Data_length = [1024, 512, 256, 128]
# parity_bits = np.array([np.log2(Data_length[i]) for i in range(len(Data_len))]) + 1    # [10 + 1, 9 + 1, 8 + 1, 7 + 1]
# KEY_LENGTH =  Data_length - parity_bits(Data_length)

# eve_presence = True
# sl = 1
# ch_noise = 2e-4
# error_threshold = 0.15
# DATA_LENGTH = 2**(11 - sl)
# key_size = 511
# eve_presence = 0 #'Random'

DATA_LENGTH = key_size    # In ./bb84_reservoir.ipynb
KEY_LENGTH = int(DATA_LENGTH - np.ceil(np.log2(DATA_LENGTH)).astype(int) - 1)
Unprocessed_key_len = 3*DATA_LENGTH

# Preparation for encoding
random.seed(0)    # Seed the random number generator. This will be used as our "coin flipper"

# order = np.ceil(np.log2(Unprocessed_key_len)).astype(int)
order = int(np.log2(Unprocessed_key_len))
dim = int(2**(order/2))
print(f"{Unprocessed_key_len = }, {KEY_LENGTH = }, {DATA_LENGTH = }, {order = }")

Unprocessed_key_len = 1022, KEY_LENGTH = 501, DATA_LENGTH = 511, order = 9


## 2.1 Alice

In [12]:
# Generating a random string of bits
# If KEY_RESERVOIR(ALICE, BOB) exists, then alice_bits = KEY_RESERVOIR[:KEY_LENGTH]. Break the iteration here.

alice_bits = generate_random_bits(Unprocessed_key_len)
alice_bases = generate_random_bases(Unprocessed_key_len) # Alice randomly chooses a basis for each bit.

print(f" Alice uncorrected {block(alice_bits, order)}")

 Alice uncorrected key size (= 1022) not an exponent of 2 : 1101111110010010100110111011100010110100000100110110101101101000011000000110011111010110001001011000000000010100000010001010001110010010111000001101001111110010101001111011100010001110110010101100111101000011100000000110011001101010010000111011110011000111101001001110101001001111100111101011011001000000100111101010101010001001110000001110010111111000111001011100000001011001110001011110000111011111100001100110100110111001000010101100110101100011101100011000011001100100100111011101001011000100000001100001011100011111100010100010001110001000111011100110110111011100010011011001101101010110010101101011001001111011100001110000111000111111000011101111011110110100011100010100011001001111011101010001111110010010000100010111101111101100001010001010101001111100010010011111110010011101111000000101011000100111101011111000001001010011010100011011100011001000001000011010011110001110101011100010110110101001010000001001100110110111011100110000

### 2.1.1 Quantum Encoding : 
**Encode the states into quantum circuits**

In [14]:
# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)

# 3. Quantum Signal Channel
This part can also be simulated by brute forcing 1-2 bit error at random.

## 3.1 Eve
0 : Not present,    1 : present

In [15]:
if eve_presence == 'Random': eve = random.randint(0, 1)
else: eve = int(eve_presence)
    
label = 'Eve' if eve else 'Alice'
print(label)

Alice


In [16]:
qubits_received = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]    # Initializing the circuit
errors_recorded = np.array([0, 0])    # Will keep track of the errors INJECTED deliberately by the algorithm

if eve : 
    #print("Eve Present!")
    qubits_intercepted = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]
    
    errors_recorded = NoisyChannel(encoded_qubits, qubits_intercepted, 'Alice', errors_recorded, noise = ch_noise) ##Eve intercepts noisy states     

    eve_bases = generate_random_bases(Unprocessed_key_len) # Generate a random set of bases
    eve_bits = measure(qubits_intercepted, eve_bases) # Measure the qubits
    
    # Eve encodes her decoy qubits and sends them along the quantum channel    
    encoded_intercepted_qubits = encode(eve_bits, eve_bases)    
    errors_recorded = NoisyChannel(encoded_intercepted_qubits, qubits_received, 'Eve', errors_recorded, noise = ch_noise) ## Eve sends noisy states to Bob

else : 
    errors_recorded = NoisyChannel(encoded_qubits, qubits_received, 'Alice', errors_recorded, noise = ch_noise) ## Alice sends noisy states to Bob


## 3.2 Bob

In [18]:
bob_bases = generate_random_bases(Unprocessed_key_len) # Bob randomly chooses a basis for each bit.

# Measurement
bob_bits = measure(qubits_received, bob_bases)

# print(f" {type(bob_bits[0]) = },  {type(alice_bits[0]) = }")

 1. Type of measured bits : <class 'str'>
 2. Type of measured bits : <class 'str'>
measured bits = '01001101010110101101011010001100101001011011001111100010011000100111000111100101100111100010011110011000000001000001001010101010100110101111000110110010011000111010011100111000100110101001001011010110011000011010100001110110011110100010100010000100011101110011110011001011010001111101111000010001010100101000111010111010010011110000010011000101110100010011110110010000010010011010010111000100100111001000001001101001101010010100100011110110011000110011100011101111011010001011110111010000110101000000011011111111100111110000011000010010100000001110101001101101000101010101110110100111100101100101001010101110000110011001011100101110011001101000111010110101111100100111100101011100001011010100010100111111001110010101100101101111011011000011100110100110011110100000101101100100100001011010100000000010001011111000110010101010011101010101100111111001100110100010000100001111000001001011100100101111101

# 4. Public Interaction Channel

* On completion of this step, the length of bits will cut down to half of the original size.
* Alice can share a string suggesting in which order to use the received bits through QSC. 
* Alice can announce the PARITY_DICT (after sifting)

## 4.1 Sifting

In [ ]:
BROADCAST = alice_bases    # Alice tells Bob which bases she used. BROADCAST uses classical channel

# Store the indices of the bases they share in common
common_bases = [i for i in range(Unprocessed_key_len) if bob_bases[i] == BROADCAST[i]]
bob_bits = [bob_bits[index] for index in common_bases]
bob_bits = ''.join(bob_bits)

print(f"\nAlice sent {block(alice_bits, order)} \n\nBob measured {block(bob_bits, order)}")

BROADCAST = common_bases    # Bob tells Alice which bases they shared in common

alice_bits = [alice_bits[index] for index in BROADCAST]    # Alice keeps only the bits they shared in common
alice_bits = ''.join(alice_bits)

## 4.2 Reconciliation 
(Comparision -- Spotting)
**Key size now reduced to half the original size**

In [33]:
# # What if Eve had listened to the previous round of key share and not this one? The remaining block would then be of mixed errors.
# # Therefore, if Eve is suspected, delete the entire lot of keys

# KEY_RESERVOIR_ALICE = np.concatenate(alice_bits[KEY_LENGTH])
# KEY_RESERVOIR_BOB = np.concatenate(bob_bits[KEY_LENGTH])

# alice_bits = KEY_RESERVOIR_ALICE[:KEY_LENGTH]
# bob_bits = KEY_RESERVOIR_BOB[:KEY_LENGTH]

# KEY_RESERVOIR_ALICE = KEY_RESERVOIR_ALICE[KEY_LENGTH:]
# KEY_RESERVOIR_BOB = KEY_RESERVOIR_BOB[KEY_LENGTH:]

In [34]:
print(len(alice_bits))

521


In [35]:
sample = len(alice_bits)//3    # len(alice_bits) >= 3
errors_detected = 0

for _ in range(sample):
    bit_index = random.randrange(len(alice_bits)) 
    
    if alice_bits[bit_index] != bob_bits[bit_index]:  errors_detected += 1    #calculating errors
        
    alice_bits = alice_bits[:bit_index] + alice_bits[bit_index+1 :] #removing tested bits from key strings
    bob_bits = bob_bits[:bit_index] + bob_bits[bit_index+1 :]

# order = np.ceil(np.log2(len(alice_bits))).astype(int)
order = Order(alice_bits)

## 4.3 QBER

In [37]:
# if errors_detected < 2:
#     hamming()

# else : cascade

In [38]:
print(f' Errors inflicted[bit, phase] : {errors_recorded}, Errors detected(total) : {errors_detected}, {sample = }')

 Errors inflicted[bit, phase] : [1 0], Errors detected(total) : 0, sample = 173


In [15]:
### QBER should be ~ 0.5 (instead of ~0.25) in presence of Eve, because the sample size is 1/3 of the bits AFTER sifting.

QBER = round(errors_detected/sample, 5) # calculating QBER and saving the answer to two decimal places
print(f"{QBER = }")

flag = 0

print(f"\n Error Threshold : {error_threshold}")

if QBER > error_threshold:
    num_keys += 1
    key = ''
    I -= 1
    raise RuntimeError('\n Eve{} detected'.format('' if eve else ' Falsely'))
    # print(""" Key not secure. Aborting protocol...
    # \r NO NEED FOR PROCEEDING TO ERROR CORRECTION \n\n\n\n""".format('' if eve else ' Falsely'))
    # elif eve and eve_presence : input(" Stuck in infinite loop ")

else :
    print(" Key is secure")
    flag = 1
    if eve : print(' Eve went unnoticed : ', end = " ")
    else : print(' Eve not present : ', end = " ")

    # if not flag : print(f" \n NO NEED FOR PROCEEDING TO ERROR CORRECTION \n\n\n\n")

    # KEY_RESERVOIR = np.concatenate(KEY_RESERVOIR, alice_bits[KEY_LENGTH:]


NameError: name 'errors_detected' is not defined

# 5. Error Correction

**If the QBER is below a certain threshold, proceed to error correction, else ABORT**

### 5.0 Creating some dummy data for Alice

## 5.1 Encoding parity bits in Alice's key

**alice_bits vs alice_block** : alice_bits contains the secure key (of length : key_len), whereas alice_block contains the redundant fillers(if any) alongwith the parity embedding

In [42]:
# After Sifting
PARITY_DICT, _ = parity(order)   # Returns empty PARITY_DICT, bin_rep

alice_block = create_parity_block(alice_bits, order, PARITY_DICT)    # Encodes parity on the block

loc = 319

 Hamming :  2 errors found
 err_count = 2, loc = 319, binary_rep = '100111111'

 Hamming Results :  (2, 319, '100111111')
 Uncorrected bit string(Order is odd, can't project to a block) : 



In [43]:
BROADCAST = PARITY_DICT

In [44]:
bob_block = create_parity_block(bob_bits, order, BROADCAST)
### Both blocks have been created, now the hamming protocol can be applied

loc = 0

 Hamming :  No errors found
 err_count = 0, loc = 0, binary_rep = '000000000'

 Hamming Results :  (0, 0, '000000000')
 Uncorrected bit string(Order is odd, can't project to a block) : 



512

In [45]:
err_count, loc, binary_rep = hamming(bob_block, order)

loc = 0

 Hamming :  No errors found
 err_count = 0, loc = 0, binary_rep = '000000000'
512 [1 1 1 0 1 1 1 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 1 0 0 1 0
 1 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 0
 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 1 1 1
 1 0 0 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 1 0
 0 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1 1 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 1
 1 0 0 1 1 0 0 1 0 1 0 1 1 0 1 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 1 1 1 1 0 0
 1 1 0 1 1 1 1 0 0 1 0 1 0 1 1 0 1 0 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1 1 1 0
 0 0 0 1 0 1 1 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 1 1 0 0 0
 1 1 1 1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 1 0 1 1 0 0 0 1 0 1 1 1
 0 0 1 1 1 1 1 1 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 

# Bob

In [47]:
print(f"Error counts : {err_count} loc : {loc} ({binary_rep})")
total_errors_recorded = errors_recorded[0] + errors_recorded[1]

if err_count != 0 :
    try : 
        if err_count == 1: bob_bits[loc] = np.mod(bob_bits[loc] + 1, 2)

    except :
        raise KeyError('Location Invalid')

print(f"\nAlice { block(alice_block, order) } \n\nBob { block(bob_block, order) }")

if all(bob_block) == all(alice_block) : 
    print(f"\n Bob bits and alice bits are same")

Error counts : 0 loc : 0 (000000000)

Alice bit string(Order is odd, can't project to a block) : 
 

Bob bit string(Order is odd, can't project to a block) : 


 Bob bits and alice bits are same


(array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
        1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1], dtype=uint8),
 array([0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1], dtype=uint8))

## Final Key : 

In [ ]:
key = ""
for bit in alice_bits:    # Or bob_bits, since both should be the same
    key += bit

print(key)

**Generate alphanumeric keys from the sequences of 0's and 1's**

If you need keys that are made of letters (e.g. for tokens or other regular use), you should group the key into octets,  
create a bytearray of octets and then encode each octet according to ASCII.

Please note that this shortens they key and to work properly quite a key of at least 8 bit is needed.